In [30]:
from mica_text_coref.coref.seq_coref import data
from mica_text_coref.coref.seq_coref import data_util
from mica_text_coref.coref.seq_coref import util

import time
import torch
import tqdm

In [2]:
train_dataset = data_util.load_tensors(("/home/sbaruah_usc_edu/"
    "mica_text_coref/data/tensors/longformer_seq_tensors_512/train/"),
    device="cpu")
(token_ids, mention_ids, label_ids,
    attn_mask, global_attn_mask, doc_ids) = train_dataset.tensors

In [27]:
def convert_tensor_to_cluster(tensor: torch.LongTensor) -> set[data.Mention]:
    """Find the set of mentions from the annotated tensor"""
    cluster: set[data.Mention] = set()
    i = 0
    while i < len(tensor):
        if tensor[i] == 1:
            j = i + 1
            while j < len(tensor) and tensor[j] == 2:
                j += 1
            mention = data.Mention(i, j - 1)
            cluster.add(mention)
            i = j
        else:
            i += 1
    return cluster

def fast_convert_tensor_to_cluster(
    tensor: torch.LongTensor) -> set[data.Mention]:
    """Find the set of mentions from the annotated tensor"""
    cluster: set[data.Mention] = set()
    n = len(tensor)
    for i in torch.where(tensor == 1)[0]:
        i = i.item()
        j = i + 1
        while j < n and tensor[j] == 2:
            j += 1
        mention = data.Mention(i, j - 1)
        cluster.add(mention)
    return cluster

In [31]:
start_time = time.time()

for labels, attn in tqdm.tqdm(zip(label_ids, attn_mask), total=len(label_ids)):
    convert_tensor_to_cluster(labels[attn == 1.])

time_taken_str = util.convert_float_seconds_to_time_string(
    time.time() - start_time)
print(time_taken_str)

100%|██████████| 22601/22601 [01:19<00:00, 285.86it/s]

0h 1m 19s


In [32]:
start_time = time.time()

for labels, attn in tqdm.tqdm(zip(label_ids, attn_mask), total=len(label_ids)):
    fast_convert_tensor_to_cluster(labels[attn == 1.])

time_taken_str = util.convert_float_seconds_to_time_string(
    time.time() - start_time)
print(time_taken_str)

100%|██████████| 22601/22601 [00:03<00:00, 7456.34it/s]

0h 0m 3s
